# Building a ReAct Agent from Scratch

**Learning Objectives:**
- Understand how LLM API calls work
- Implement the ReAct (Reasoning + Acting) framework
- Connect an agent to real tools (Grasshopper)

**Time:** ~30 minutes follow-along

---

## Part 1: Setup

In [ ]:
# Core imports
import re      # Regular expressions - used in Part 5 to parse LLM output
import json
import os

# For LLM calls - Groq provides fast inference with generous free tier
from groq import Groq

# For MCP connection (Grasshopper tools) - httpx handles SSE streaming
import httpx

In [ ]:
# Configure your API key
# Get your FREE API key at: https://console.groq.com/keys
# Option 1: Set environment variable GROQ_API_KEY
# Option 2: Replace the string below

API_KEY = os.environ.get("GROQ_API_KEY", "your-api-key-here")

# Initialize the Groq client
client = Groq(api_key=API_KEY)

# Quick test - Groq is FAST (100+ tokens/sec!)
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": "Tell me a short haiku about computational design."}],
    temperature=0.7
)
print(response.choices[0].message.content)

### Provider Abstraction

The function below wraps our LLM calls. To switch providers (OpenAI, Gemini, Anthropic, local models),
you only need to change this one function. The rest of the agent code stays the same.


In [ ]:
def call_llm(messages: list[dict], temperature: float = 0) -> str:
    """
    Call the LLM with a list of messages.

    To switch providers, modify this function only.
    Currently using: Groq (Llama 3.3 70B)

    Args:
        messages: List of {"role": "user"|"assistant"|"system", "content": "..."}
        temperature: 0 = deterministic, 1 = creative

    Returns:
        The assistant's response text
    """
    # Groq uses OpenAI-compatible API - messages format works directly!
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages,
        temperature=temperature,
        max_tokens=4096
    )
    return response.choices[0].message.content

---

## Part 2: The Agent Class

An agent is simply:
1. A **system prompt** that defines its behavior
2. A **message history** that tracks the conversation
3. A **method to call the LLM** and append responses

Let's build it from scratch:

In [ ]:
class Agent:
    """
    A simple conversational agent that maintains message history.

    The agent follows whatever behavior is defined in its system prompt.
    """

    def __init__(self, system_prompt: str = ""):
        """Initialize the agent with an optional system prompt."""
        # TODO: Store the system prompt
        # TODO: Initialize an empty messages list
        # TODO: If system_prompt is provided, add it to messages with role "system"
        pass

    def __call__(self, user_message: str) -> str:
        """
        Send a message to the agent and get a response.

        This method:
        1. Adds the user message to history
        2. Calls the LLM with full history
        3. Adds the response to history
        4. Returns the response
        """
        # TODO: Append user message to self.messages with role "user"
        # TODO: Call call_llm(self.messages) to get the response
        # TODO: Append the response to self.messages with role "assistant"
        # TODO: Return the response
        pass

    def reset(self):
        """Clear conversation history (keeps system prompt)."""
        # TODO: Reset self.messages to empty list
        # TODO: If system_prompt exists, add it back to messages
        pass

In [ ]:
# Quick test - a simple chatbot
test_agent = Agent("You are a helpful assistant. Be concise.")
print(test_agent("What is 2 + 2?"))
print(test_agent("What did I just ask you?"))  # Tests memory

---

## Part 3: Connecting to Real Tools (Grasshopper MCP)

Now we'll connect our agent to **real tools** that can control Grasshopper.

The MCP (Model Context Protocol) server exposes these tools:
- `list_python_scripts` - Find script components on the canvas
- `get_python_script` - Read a script's code
- `edit_python_script` - Write/modify code
- `get_python_script_errors` - Check for compilation errors

**Important:** Make sure Grasshopper is running with the MCP server active!

In [ ]:
# MCP Server Configuration
# The Grasshopper MCP uses HTTP + SSE (Server-Sent Events) protocol
MCP_URL = "http://127.0.0.1:8089/mcp"  # Default endpoint

import httpx

def call_mcp_tool(tool_name: str, arguments: dict = None) -> dict:
    """
    Call a tool on the Grasshopper MCP server.
    
    The MCP uses SSE for responses, so we need to stream and parse the events.
    """
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": tool_name,
            "arguments": arguments or {}
        }
    }
    
    try:
        # Use httpx with streaming to handle SSE response
        with httpx.Client(timeout=30) as client:
            with client.stream("POST", MCP_URL, json=payload) as response:
                # Read SSE events
                for line in response.iter_lines():
                    # SSE format: "data: {json}"
                    if line.startswith("data: "):
                        data = json.loads(line[6:])
                        # Extract the result from MCP response
                        if "result" in data:
                            return data["result"]
                        return data
                    elif line.strip() and not line.startswith(":"):
                        # Try parsing as plain JSON
                        try:
                            return json.loads(line)
                        except:
                            pass
        return {"error": "No response received"}
    except Exception as e:
        return {"error": str(e)}

In [ ]:
# Fetch available tools from MCP server
def get_available_tools() -> list:
    """Fetch tool definitions from the MCP server."""
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/list",
        "params": {}
    }
    
    try:
        with httpx.Client(timeout=30) as client:
            with client.stream("POST", MCP_URL, json=payload) as response:
                for line in response.iter_lines():
                    if line.startswith("data: "):
                        data = json.loads(line[6:])
                        if "result" in data and "tools" in data["result"]:
                            return data["result"]["tools"]
        return []
    except Exception as e:
        print(f"Error fetching tools: {e}")
        return []

# Get tools and filter to the ones we want for this tutorial
ALL_TOOLS = get_available_tools()

# For this tutorial, we focus on Python script tools
TUTORIAL_TOOLS = [t for t in ALL_TOOLS if t["name"] in [
    "List_Python_Scripts",
    "Get_Python_Script", 
    "Edit_Python_Script",
    "Get_Python_Script_Errors"
]]

print(f"Found {len(ALL_TOOLS)} total tools, using {len(TUTORIAL_TOOLS)} for this tutorial:")
for tool in TUTORIAL_TOOLS:
    print(f"  - {tool['name']}: {tool['description'][:60]}...")

In [ ]:
# Test: Call a tool directly
result = call_mcp_tool("List_Python_Scripts")
print("Python scripts on canvas:", result)

---

## Part 4: The ReAct Framework

**ReAct** = **Re**asoning + **Act**ing

The key insight: LLMs can follow a structured loop if we tell them to:

1. **Thought** - Reason about what to do next
2. **Action** - Call a tool with specific input
3. **PAUSE** - Stop and wait for the tool result
4. **Observation** - Receive the tool's output
5. **Repeat** until ready to give final **Answer**

The magic is in the **prompt**. Let's build it:

In [ ]:
# Build ReAct prompt with Anthropic best practices + Grasshopper context

def build_react_prompt(tools: list) -> str:
    """Build a ReAct prompt from MCP tool definitions."""
    
    # Format tools as simple list
    tool_docs = []
    for tool in tools:
        name = tool["name"]
        desc = tool["description"].split('.')[0]  # First sentence only
        params = tool.get("inputSchema", {}).get("properties", {})
        param_names = list(params.keys())
        tool_docs.append(f"- {name}: {desc}. Parameters: {param_names if param_names else 'none'}")
    
    tools_section = "\n".join(tool_docs)
    
    return f"""You are a Grasshopper Python scripting assistant that creates geometry.

<context>
You write Python scripts for Grasshopper (Rhino's visual programming tool).
Scripts run inside a Python component with predefined output variables.
</context>

<grasshopper_rules>
- Output geometry by assigning to variable 'a': a = my_points
- Import Rhino.Geometry as rg for geometry types
- Use rg.Point3d(x,y,z) for points, rg.Line() for lines, etc.
- Lists of geometry display as multiple objects
- Always assign final result to 'a' or it won't display!
</grasshopper_rules>

<tools>
{tools_section}
</tools>

<workflow>
1. Thought: reason about what to do
2. Action: call a tool, then PAUSE
3. Wait for Observation
4. After editing code, always check for errors
5. Only give Answer when code works
</workflow>

<format>
Action: ToolName {{"param": "value"}}
</format>

<example>
User: Create a spiral of 10 points

Thought: Find the script component first.
Action: List_Python_Scripts
PAUSE

Observation: [{{"id": "abc-123", "name": "Task"}}]

Thought: Write code for a spiral. Must assign to 'a' for output.
Action: Edit_Python_Script {{"componentId": "abc-123", "code": "import Rhino.Geometry as rg\\nimport math\\n\\npoints = []\\nfor i in range(10):\\n    angle = i * 0.5\\n    x = math.cos(angle) * i\\n    y = math.sin(angle) * i\\n    points.append(rg.Point3d(x, y, 0))\\n\\na = points"}}
PAUSE

Observation: Script updated

Thought: Check for errors.
Action: Get_Python_Script_Errors {{"componentId": "abc-123"}}
PAUSE

Observation: No errors

Answer: Created spiral of 10 points, assigned to output 'a'.
</example>
"""

REACT_PROMPT = build_react_prompt(TUTORIAL_TOOLS)
print(f"Prompt built ({len(REACT_PROMPT)} chars)")
print(REACT_PROMPT)

### Why This Prompt Works

Notice the key elements:
1. **Role definition** - "You are a Grasshopper scripting assistant"
2. **Loop structure** - Explicit Thought -> Action -> PAUSE -> Observation cycle
3. **Tool descriptions** - Exact syntax with examples
4. **Workflow rules** - "ALWAYS check for errors"
5. **Example session** - Shows the expected format

The LLM follows this because it's trained to follow instructions.
The more specific and structured your prompt, the more reliable the behavior.

---

## Part 5: The ReAct Execution Loop

Now we need code that:
1. Parses the LLM's output to find Actions
2. Executes the corresponding tool
3. Feeds the result back as an Observation
4. Repeats until we get an Answer

In [ ]:
# Parse actions from LLM response
# Format: "Action: ToolName" or "Action: ToolName {"param": "value"}"
ACTION_PATTERN = re.compile(r'^Action:\s*(\w+)\s*(\{.*\})?$', re.MULTILINE)

def parse_action(response: str) -> tuple[str, dict] | None:
    """
    Parse an action from the LLM response.
    
    Expected format in response:
        Action: ToolName {"param": "value"}
    
    Returns: (tool_name, arguments_dict) or None if no action found
    """
    # TODO: Use ACTION_PATTERN.search(response) to find an action
    # TODO: If match found:
    #   - Extract tool_name from match.group(1)
    #   - Extract args_str from match.group(2) (may be None)
    #   - If args_str exists, parse it with json.loads()
    #   - Return (tool_name, args)
    # TODO: If no match, return None
    pass

In [ ]:
# Execute tool actions via MCP
def execute_action(tool_name: str, args: dict) -> str:
    """
    Execute a tool and return the result as a string.
    
    Args:
        tool_name: Name of the MCP tool to call
        args: Dictionary of arguments for the tool
    
    Returns:
        String representation of the result (for feeding back to LLM)
    """
    # TODO: Call call_mcp_tool(tool_name, args) to get result
    # TODO: Format the result as a string:
    #   - If result is a dict with "error" key, return f"Error: {result['error']}"
    #   - If result is a dict or list, return json.dumps(result, indent=2)
    #   - Otherwise return str(result)
    pass

In [ ]:
# The ReAct execution loop
def query(question: str, max_turns: int = 10, verbose: bool = True) -> str:
    """
    Run the ReAct loop to answer a question.
    
    The loop:
    1. Send question/observation to agent
    2. Check if response contains "Answer:" (without PAUSE) -> return it
    3. Parse action from response
    4. Execute action and get observation
    5. Feed observation back and repeat
    
    Args:
        question: The user's question/request
        max_turns: Maximum number of reasoning turns
        verbose: Whether to print each turn
    
    Returns:
        The final answer from the agent
    """
    # TODO: Create an Agent with REACT_PROMPT
    # TODO: Set next_prompt = question
    # TODO: Loop for max_turns:
    #   - Call agent(next_prompt) to get response
    #   - If verbose, print the turn number and response
    #   - Check if "Answer:" in response AND "PAUSE" not in response
    #       -> If so, extract and return the answer
    #   - Call parse_action(response) to get action
    #   - If action found:
    #       -> Execute it with execute_action()
    #       -> Set next_prompt = f"Observation: {observation}"
    #   - If no action found:
    #       -> Set next_prompt = "Continue with an Action or provide your Answer."
    # TODO: Return "Max turns reached." if loop exhausts
    pass

---

## Part 6: Let's Run It!

Now for the exciting part. Let's ask our agent to create geometry in Grasshopper.

**Make sure:**
1. Grasshopper is open
2. You have a Python 3 Script component on the canvas
3. The MCP server is running

In [ ]:
# THE MAIN EVENT
# Change this prompt to create different geometry!

user_prompt = """
Create a spiral staircase including steps
"""

result = query(user_prompt)
print("\n" + "="*50)
print("FINAL ANSWER:")
print("="*50)
print(result)

---

## What You Learned

1. **LLM API calls** are just HTTP requests with message history
2. **ReAct framework** = Thought + Action + Observation loop
3. **The prompt is everything** - structured instructions create structured behavior
4. **Tools extend capabilities** - connect LLMs to real systems via MCP

## Next Steps

- Read the original ReAct paper: Yao et al., 2022
- Explore the Grasshopper MCP documentation
- Try connecting other tools (web search, file system, etc.)